### Exploring the dataset. Credits for this notebook goes to [Reynaldo][1]


  [1]: https://www.kaggle.com/reynaldo

In [ ]:
 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
#print(check_output(["ls", "../input/train.csv"]).decode("utf8"))

In [ ]:
data = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
y_train = data["price_doc"]
X_train = data.drop(["id", "timestamp","price_doc"], axis=1)
X_test = test.drop(["id", "timestamp"], axis=1)

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)

In [ ]:
from sklearn import model_selection, preprocessing
import xgboost as xgb

In [ ]:
for c in X_train.columns:
    if X_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[c].values)) 
        X_train[c] = lbl.transform(list(X_train[c].values))
        
for c in X_test.columns:
    if X_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_test[c].values)) 
        X_test[c] = lbl.transform(list(X_test[c].values))

In [ ]:
xgb_params = {
    'eta': 0.06,
    'max_depth': 10,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=2000, early_stopping_rounds=40,
    verbose_eval=100, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [ ]:
id_test = test.id
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

In [ ]:
output.to_csv('submission2.csv', index=False)